In [5]:
from os.path import exists
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.pylab as pylab

from soursop.sstrajectory import SSTrajectory
import mdtraj as md

# define this so we can read in nucleotides
NA_EXTENSION = ['D5P', 'DPC',  'DPU',  'DPT',  'DPA',  'DPG',  'R5P',  'RPC' , 'RPU',  'RPT',  'RPA',  'RPG']

# molar to nanomolar conversion factor
M2nM=1e9

import numpy as np


In [6]:

# ......................................................
#
def build_traces(COM_distances, bound_length, cutoff):
    """
    Function that assesses bound and unbound states based on contigous stretches of
    the simulation where the centers of mass are conistently below some threshold.
    
    In particular, $bound_length defines the number of consecutive frames which must 
    be below some threshold for a simulation to be considered bound. $cutoff defines what
    this distance threshold is. For example, if bound_length=20 (default) we only consider
    regions of the inter-molecular COM distance. 
    
    Parameters
    -------------
    COM_distances : np.ndarray
        Array of inter-molecular center of mass (COM) distances. Expect one value
        per frame.
        
    bound_length : int
        Number of consecutive frames required to designate an interaction as 'bound'
        
    cutoff : float
        Distance (in Angstroms) used to define if two molecules are bound or not. Note
        this is the COM distance, not the minimum contact distance (!).
        
        
    Returns
    -----------
    tuple
        Returns a tuple with three elements
        
        # fraction bound
        [0] - float      - the fraction bound of this simulation
        
        # bound_trace
        [1] - np.ndarray - shape=(2,n), where the two columns are the
                           index of a bound frame and the distance in 
                           that frame
                           
        # unbound_trace
        [2] - np.ndarray - shape=(2,n), where the two columns are the
                           index of a bound frame and the distance in 
                           that frame
    
    """

    # initialize some variables
    bound_trace = []
    unbound_trace = []
    inside = False

    all_bound_idx = set([])
    all_unbound_idx = set([])

    idx = bound_length-1
    
    # cycle over the bound_length-thed index to the end of
    # the COM array
    for idx in range(bound_length, len(COM_distance)):

        # get the distance at the idx position
        i = COM_distance[idx]

        # define the preceding bound_length-sized set of
        # frames (i.e. the 'history' is the set frames
        # that come $bound_length before the current 
        # index
        history = COM_distance[idx-bound_length:idx]

        # if we're already 'inside' a bound part of the
        # trajectory
        if inside:  
            
            # if under the cutoff then carry on, we've found
            # additional frames in a contigous set of frames 
            # that are bound
            if i < cutoff:            
                bound_trace.append([idx,i])
                all_bound_idx.add(idx)
                
            # if we're above the cutoff then this means an unbinding
            # event happens
            else:
                inside = False
                
        # if we're not currently inside a bound set of frames
        else:
            
            # if this frame is bound
            if i < cutoff:
                
                # if were the last set of frames all under the reshold
                if np.max(history) < cutoff:
                    
                    # if yes we're insidew
                    inside = True

                    for tmp_idx, tmp_i in enumerate(history):
                        local_idx = idx-(bound_length - tmp_idx)
                        bound_trace.append([local_idx, tmp_i])
                        all_bound_idx.add(local_idx)
                        
                    bound_trace.append([idx,i])
                    all_bound_idx.add(idx)


    # now, having classufued every frame, we can go through all the frames and 
    # if they're not in one of the all_bound_idx set they must be all unbound
    for idx in range(bound_length, len(COM_distance)):
        i = COM_distance[idx]
        if idx not in all_bound_idx:
            unbound_trace.append([idx,i])
            all_unbound_idx.add(idx)
        
    bound_trace = np.array(bound_trace).transpose()
    unbound_trace = np.array(unbound_trace).transpose()
    
    # calculate the fraction bound
    fraction_bound = len(all_bound_idx)/(len(all_unbound_idx)+len(all_bound_idx))
 
    return (fraction_bound, bound_trace, unbound_trace)

# ......................................................
#
def naive_Kd(p_bound, L):
    """
    Calculate the binding affinity using the so-called niave 
    assumption from Jost et al. [1]
        
    Parameters
    --------------
    p_bound : float
        Fraction of the simulation in which the two molecules 
        are bound. How one defines 'bound' is often the hard 
        question. This is unitless (i.e. is a fraction)
        
    L : float
        Box dimensions (in nanometers)
        
    Returns
    -------------
    float
    
        Returns the kD in molar units (mol/L)
        
    Reference
    --------------
    [1] - Jost Lopez, A., Quoika, P. K., Linke, M., Hummer, G., 
    & Köfinger, J. (2020). Quantifying Protein–Protein Interactions 
    in Molecular Simulations. The Journal of Physical Chemistry. B, 
    124(23), 4673–4685.

    """
    
    # Avogadro's number - recall that 1/NA gives us units of mol
    # because NA/NA = 1 mol 
    N_A = 6.02214076*1e23
    
    # Volume of simulation box in L (convert to cubic meters and)
    # then to liters
    V = 1e3*((L*1e-09)**3)
    
    # kd (in mol/L) determined from p_bound
    kd = ((1-p_bound)**2) / (N_A*V*p_bound)
    
    return kd



In [10]:
# this empirically threshold defines 
len2cutoff = {}
len2cutoff[10] = 42
len2cutoff[15] = 44
len2cutoff[20] = 46
len2cutoff[25] = 48
len2cutoff[30] = 50
len2cutoff[35] = 60
len2cutoff[40] = 65

len2cutoff[12] = 43
len2cutoff[17] = 45

In [33]:
RNA_lengths = [10, 12, 15, 17, 20, 25, 30, 35, 40]
construct_names = ['NTDRBD', 'NTD', 'RBD']
n_reps = 3
bound_length = 5

# box size in nm
L = 30

all_KA = {}

for rep in range(1,n_reps+1):
    all_KA[rep] = {}
    for c in construct_names:
        all_KA[rep][c] = {}
        for rna_len in RNA_lengths:
            
            dirname = f'/work/alstonj/2022/Binding/FINAL/{c}/{rna_len}/{rep}'
            
            CM_ss = SSTrajectory(f'{dirname}/comtraj.xtc', f'{dirname}/comtraj.pdb',extra_valid_residue_names=['CM'])
            
            COM_distance = CM_ss.get_interchain_distance(0,1,0,0)
            
            tmp = build_traces(COM_distance, cutoff=len2cutoff[rna_len], bound_length=bound_length)
            
            fraction_bound = tmp[0]
            all_KA[rep][c][rna_len] = 1/(naive_Kd(fraction_bound, L)*M2nM)
            
            


In [40]:
mean_KA = {}
std_err_KA = {}
for c in construct_names:
    mean_KA[c] = {}
    std_err_KA[c] = {}
    for rna_len in RNA_lengths:
        mean_KA[c][rna_len] = np.mean([all_KA[1][c][rna_len], all_KA[2][c][rna_len],all_KA[3][c][rna_len]])
        
        # note we calculate 'error' here as standard deviation as opposed to standard error of mean because
        # std error of mean is incredibly small (like, the error bars are not visible for any plot) because we
        # have a LOT of simulation data (~250,000 frames per protein/RNA combo)
        std_err_KA[c][rna_len] = np.std([all_KA[1][c][rna_len], all_KA[2][c][rna_len],all_KA[3][c][rna_len]])
        


In [41]:
mean_KA_norm = {}
std_err_KA_norm = {}

err_ref    = std_err_KA['NTDRBD'][25]
mean_ref   = mean_KA['NTDRBD'][25]

for c in construct_names:
    mean_KA_norm[c] = {}
    std_err_KA_norm[c] = {}
    for rna_len in RNA_lengths:
        
        err_target  = std_err_KA[c][rna_len]
        mean_target = mean_KA[c][rna_len]
                
        mean_KA_norm[c][rna_len] = mean_target/mean_ref
        std_err_KA_norm[c][rna_len] = np.sqrt((err_target/mean_target)**2 + (err_ref/mean_ref)**2)*mean_KA_norm[c][rna_len]
        
        

In [45]:
for c in construct_names:
    with open(f'{c}_normalized_Ka.csv','w') as fh:
        fh.write('#RNA length, KA-star, KA-star std\n')
        for rna_len in RNA_lengths:
            fh.write(f'{rna_len}, {mean_KA_norm[c][rna_len]}, {std_err_KA_norm[c][rna_len]}\n')

In [47]:
for c in construct_names:
    with open(f'{c}_raw_Ka.csv','w') as fh:
        fh.write('#RNA length, KA (nM), KA std (nM)\n')
        for rna_len in RNA_lengths:
            fh.write(f'{rna_len}, {mean_KA[c][rna_len]}, {std_err_KA[c][rna_len]}\n')